In [ ]:
import numpy as np
import igl
import meshplot as mp

In [ ]:
# Utility function to generate a tet grid
# n is a 3-tuple with the number of cell in every direction
# mmin/mmax are the grid bounding box corners

def tet_grid(n, mmin, mmax):
    nx = n[0]
    ny = n[1]
    nz = n[2]
    
    delta = mmax-mmin
    
    deltax = delta[0]/(nx-1)
    deltay = delta[1]/(ny-1)
    deltaz = delta[2]/(nz-1)
    
    T = np.zeros(((nx-1)*(ny-1)*(nz-1)*6, 4), dtype=np.int64)
    V = np.zeros((nx*ny*nz, 3))

    mapping = -np.ones((nx, ny, nz), dtype=np.int64)

    index = 0
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                mapping[i, j, k] = index
                V[index, :] = [i*deltax, j*deltay, k*deltaz]
                index += 1
    assert(index == V.shape[0])
    
    tets = np.array([
        [0,1,3,4],
        [5,2,6,7],
        [4,1,5,3],
        [4,3,7,5],
        [3,1,5,2],
        [2,3,7,5]
    ])
    
    index = 0
    for i in range(nx-1):
        for j in range(ny-1):
            for k in range(nz-1):
                indices = [
                    (i,   j,   k),
                    (i+1, j,   k),
                    (i+1, j+1, k),
                    (i,   j+1, k),

                    (i,   j,   k+1),
                    (i+1, j,   k+1),
                    (i+1, j+1, k+1),
                    (i,   j+1, k+1),
                ]
                
                for t in range(tets.shape[0]):
                    tmp = [mapping[indices[ii]] for ii in tets[t, :]]
                    T[index, :]=tmp
                    index += 1
                    
    assert(index == T.shape[0])
    
    V += mmin
    return V, T

# Reading point cloud

In [ ]:
pi, v = igl.read_triangle_mesh("data/cat.off")
pi /= 10
ni = igl.per_vertex_normals(pi, v)


In [ ]:
def find_closest_point(point, points):
    return np.argmin(list(map((lambda y: np.linalg.norm(point - y)), points)))

def find_eps_point(eps, point, n):
    return point + eps * n

def find_eps(eps, point, points, n, i):
    ep_point = find_eps_point(eps, point, n)
    while find_closest_point(point, points) != i:
        eps /= 2
        ep_point = find_eps_point(eps, point)
    return ep_point


def create_point_cloud(pi, ni):
    eps_start = 0.01 * np.linalg.norm(np.max(pi, axis = 0) - np.min(pi, axis = 0))

    eps = np.zeros(pi.shape)
    pi_plus = np.zeros(pi.shape)
    pi_minus = np.zeros(pi.shape)

    p_con = []
    f_con = []
    
    for i, point in enumerate(pi):
        p_con.append(point)
        f_con.append(0)
        
        p = find_eps(eps_start, point, pi, ni[i], i)
        pi_plus[i] = p
        p_con.append(p)
        f_con.append(eps_start)
        
        p = find_eps(-eps_start, point, pi, ni[i], i)
        pi_minus[i] = p
        p_con.append(p)
        f_con.append(-eps_start)

    p_con = np.array(p_con)
    f_con = np.array(f_con)
    return p_con, f_con, pi_plus, pi_minus

# MLS function

In [ ]:
# Parameters
def get_MLS_grid_paramter(p_con):
    ma = np.ceil(np.max(p_con, axis = 0))
    mi = np.floor(np.min(p_con, axis = 0))

    m = int(np.max((np.abs(ma), np.abs(mi))))

    bbox_min = np.array(mi)
    bbox_max = np.array(ma)

    bbox_diag = np.linalg.norm(bbox_max - bbox_min)
    return bbox_min, bbox_max, bbox_diag

In [ ]:
def closest_points(point, points, h):
    distances = np.linalg.norm(point-points, axis = 1)
    points = np.where(distances < h)[0]
    return points

def set_wendland_matrix(x, points, close_points, radius):
    W = []
    for i in close_points:
        W.append(wendland_weight_func(np.linalg.norm(x - points[i]), radius))
    
    W = np.diag(W) 
    return W

def wendland_weight_func(r, h):
    return (1-r/h)**4 * (4*r/h+1) if r < h else 100

def pbf(pi, poly_degree):
    if poly_degree == 2:
        return degree_two(pi)
    elif poly_degree == 1:
        return degree_one(pi)
    return np.array([1])

def degree_one(pi):
    return np.array([1, pi[0], pi[1], pi[2]])

def degree_two(pi):
    x = pi[0]
    y = pi[1]
    z = pi[2]
    return np.array([1, x, y, z, x*y, x*z, y*z, x*x, y*y, z*z])

def create_B(ind, points, poly_degree):
    B = []
    for i in range(ind.shape[0]):
        if poly_degree == 1:
            B.append(degree_one(points[ind[i]]))
        elif poly_degree == 2:
            B.append(degree_two(points[ind[i]]))
        else:
            B.append(np.array([1]))
    return np.array(B)

def create_d(ind, f):
    d = []
    for i in ind:
        d.append(f[i])
    return np.array(d)

In [ ]:
# Generate grid n x n x n
def MLS_interpolation(p_con, f_con, n = 25, wendlandRadius = 30, poly_degree = 1):
    bbox_min, bbox_max, bbox_diag = get_MLS_grid_paramter(p_con)
    x, T = tet_grid((n, n, n), bbox_min - 0.05 * bbox_diag, bbox_max + 0.05 * bbox_diag)
    
    fx = []
    for i, xi in enumerate(x):
        close_points = closest_points(xi, p_con, wendlandRadius)
        if len(close_points) > (poly_degree + 1) ** 2:
            W = set_wendland_matrix(xi, p_con, close_points, wendlandRadius)
            B = create_B(close_points, p_con, poly_degree)
            d = create_d(close_points, f_con)
            BT = np.matrix.transpose(B)
            a = np.linalg.solve(BT@W@B, BT@W@d)
            xT = pbf(x[i], poly_degree)
            fx.append(xT@a)
        else:
            fx.append(100.0)
    fx = np.array(fx)
    return fx, x, T


def MLS_interpolation_si(p_con, f_con, n = 25, wendlandRadius = 30, poly_degree = 1):
    bbox_min, bbox_max, bbox_diag = get_MLS_grid_paramter(p_con)
    x, T = tet_grid((n, n, n), bbox_min - 0.05 * bbox_diag, bbox_max + 0.05 * bbox_diag)
    spin, dims, bin_size = create_spatial_index(p_con, 10, bbox_min, bbox_max)
    
    fx = []
    for i in range(x.shape[0]):
        points_in_range = np.array(query_spatial(spin, p_con, x[i], bin_size, bbox_min, dims, wendlandRadius))
        if len(points_in_range) > (poly_degree +1)*2:
            W = set_wendland_matrix(x[i], p_con, points_in_range, wendlandRadius)
            B = create_B(points_in_range, p_con, poly_degree)
            d = create_d(points_in_range, f_con)
            BT = np.matrix.transpose(B)
            a = np.linalg.solve(BT@W@B, BT@W@d)
            xT = pbf(x[i], poly_degree)
            fx.append(xT@a)
        else:
            fx.append(1000.0)
    fx = np.array(fx)
    return fx, x, T

In [ ]:
# Treshold fx to visualize inside outside
def get_MLS_ind_color(fx, x):
    ind = np.zeros_like(fx)
    ind[fx >= 0] = 1
    ind[fx < 0] = -1
    return ind

In [ ]:
def create_mesh_si(data, n=5, wendlandRadius = 30, poly_degree = 2):
    pi, ni = get_points(data)
    p = mp.subplot(pi, shading={'point_size':point_size}, s=[4,4,0])
    
    p_con, f_con, _, _ = create_point_cloud(pi, ni)
    red = [1,0,0]
    blue = [0,1,0]
    green = [0,0,1]
    col = np.array([green if indx %3==0 else red if indx %3==1 else blue for indx in range(p_con.shape[0])])
    mp.subplot(p_con, c=col, shading={'point_size':point_size}, s=[4,4,1], data=p)
    
    fx, x = MLS_interpolation(p_con, f_con, n, wendlandRadius, poly_degree)
    col = get_MLS_ind_color(fx, x)
    mp.subplot(x, c=col, shading={'point_size':point_size}, s=[4,4,2], data=p)
   
    v, f, _, _ = igl.marching_tets(x, T, fx, 0)
    mp.subplot(v, f, shading={'wireframe':True}, s=[4,4,3], data=p)

In [ ]:
def create_spatial_index(points, num_bins, bbox_min, bbox_max):
    bin_size = (bbox_max - bbox_min) / num_bins
    dims = np.ceil((bbox_max - bbox_min) / bin_size).astype(int)
    spin = [None] * dims[0] * dims[1] * dims[2]
    
    for i, point in enumerate(points):
        coords = ((point - bbox_min) / bin_size).astype(int)
        index = np.ravel_multi_index(coords, dims)
        
        if spin[index] is None:
            spin[index] = [i]
        else:
            spin[index].append(i)
    
    return spin, dims, bin_size


def query_spatial(spin, points, xi, bin_size, bbox_min, dims, rad):
    search_coords = ((xi - bbox_min) / bin_size).astype(int)
    min_coords = np.floor((xi - rad - bbox_min) / bin_size).astype(int)
    max_coords = np.ceil((xi + rad - bbox_min) / bin_size).astype(int)
    
    points_in_range = []
    
    grid_coords = np.meshgrid(
        np.arange(min_coords[0], max_coords[0]+1),
        np.arange(min_coords[1], max_coords[1]+1),
        np.arange(min_coords[2], max_coords[2]+1),
        indexing='ij'
    )
    
    grid_coords = np.array(grid_coords).reshape(3, -1).T
    
    valid_ind = np.all((grid_coords >=0) & (grid_coords < dims), axis=1)
    valid_coords = grid_coords[valid_ind]
    valid_ind = np.ravel_multi_index(valid_coords.T, dims)
    
    valid_cells = [cell for cell in valid_ind if spin[cell] is not None]
    
    for cell in valid_cells:
        for point in spin[cell]:
            dis = np.linalg.norm(points[point] - xi)
            if dis <= rad:
                points_in_range.append(point)
    
    return points_in_range

In [ ]:
def align_points_to_axes(points):
    centroid = np.mean(points, axis = 0)

    # Y = points - centroid
    # S = Y @ Y.T

    cent_points = points - centroid
    S = np.dot(cent_points.T, cent_points)

    eigenvalues, eigenvectors = np.linalg.eig(S)

    ind = np.argsort(eigenvalues)[::-1]
    eigenvectors = eigenvectors[:, ind]

    rot_cen_points = cent_points @ eigenvectors
    
    rot_points = rot_cen_points + centroid
    return rot_points

In [ ]:
def get_points(data):
    pi, v = igl.read_triangle_mesh(data)
    pi /= 10
    ni = igl.per_vertex_normals(pi, v)
    return pi, ni
    
def extract_max_component(v,f):
    fcf = igl.facet_components(f)
    large_component = np.argmax(np.bincount(fcf))
    large_f = f[fcf == large_component]
    return v, large_f
    
def create_mesh(data, n= 25, wendlandRadius = 30, poly_degree = 2, point_size=5, si = False, align=False):
    pi, ni = get_points(data)
    p = mp.subplot(pi, shading={'point_size':point_size}, s=[4,4,0])
    
    p_con, f_con, _, _ = create_point_cloud(pi, ni)
    if align:
        p_con = align_points_to_axes(p_con)
        
    red = [1,0,0]
    blue = [0,1,0]
    green = [0,0,1]
    col = np.array([green if indx %3==0 else red if indx %3==1 else blue for indx in range(p_con.shape[0])])
    mp.subplot(p_con, c=col, shading={'point_size':point_size}, s=[4,4,1], data=p)
    
    fx, x, T = MLS_interpolation(p_con, f_con, n, wendlandRadius, poly_degree) if not si else MLS_interpolation_si(p_con, f_con, n, wendlandRadius, poly_degree)
    col = get_MLS_ind_color(fx, x)
    mp.subplot(x, c=col, shading={'point_size':point_size}, s=[4,4,2], data=p)
   
    v, f, _, _ = igl.marching_tets(x, T, fx, 0)
    v, f = extract_max_component(v,f)
    mp.subplot(v, f, shading={'wireframe':True}, s=[4,4,3], data=p)
    

# Marching to extract surface

In [ ]:
# To complete mesh reconstruction process use following method:
# def create_mesh(data, n, wendlandRadius, poly_degree, point_size, si, align):
#     data: location of .off file containing vertices
#     n: number of cells per direction for tet grid construction
#     wendlandRadius: radius used for Wendland weights
#     poly_degree: polynomial degree for the polynomial basis function
#     point_size: size of points when displaying point clouds and grid sampling
#     si: use the spatial index for finding closest points during MLS
#     align: align point cloud to principal axes

# Uncomment following lines for model reconstruction
# create_mesh("data/cat.off", n=20, wendlandRadius=30, poly_degree=2, point_size=10, si=False, align=False)
# create_mesh("data/luigi.off", n=20, wendlandRadius=1, poly_degree=2, point_size=1, si=True, align=True)